# TM Project

## Toughts
* Should we apply the different metrics for each language pair respectively according to its best correlation?

### Importing the data

In [1]:
import pandas as pd
import numpy as np

In [2]:
#importing the data

path_corpus = "/Users/franz/Desktop/TM Project/corpus/"

ru_en = pd.read_csv(path_corpus + "ru-en/scores.csv")
de_en = pd.read_csv(path_corpus + "de-en/scores.csv")
cs_en = pd.read_csv(path_corpus + "cs-en/scores.csv")
zh_en = pd.read_csv(path_corpus + "zh-en/scores.csv")
en_zh = pd.read_csv(path_corpus + "en-zh/scores.csv")
en_fi = pd.read_csv(path_corpus + "en-fi/scores.csv")

In [3]:
de_en.head()

,source,reference,translation,z-score,avg-score,annotators
0,"Ihr Zeitlupentempo maßen sie, als sie vor Spit...",Her timeless pace measures them when they equi...,Their slow speed was measured by researchers o...,-0.345024,76.0,1
1,"Er sagte, dass die Bereiche ruhige Treffpunkte...",He said the areas offer quiet meeting points b...,He said the spaces provided calm meeting point...,0.903800,97.5,2
2,Für die Geschäftsleute an der B 27 ist es nur ...,"For businessmen at the B 27, it's only a small...",This is only a small consolation for businesse...,0.700503,94.0,1
3,Diese Fähigkeit sei möglicherweise angeboren o...,This ability may be born or developed with gen...,"This ability may be innate, or may develop as ...",-1.256572,51.5,2
4,Weil sie Wassertemperaturen um die sechs Grad ...,Because they prefer water temperatures around ...,They generally only come to the surface in win...,0.293909,87.0,2


### Data exploration

In [4]:
descriptions = ["Russian into English", "German into English", "Czech into English", "Chinese into English", "English into Chinese", "English into Finish"]

In [5]:
rows = []
zscores = []
avgscores = []
annots = []

i = 0

for element in [ru_en, de_en, cs_en, zh_en, en_zh, en_fi]:
    rows.append(element.shape[0])
    zscores.append(np.round(element["z-score"].mean(),2))
    avgscores.append(np.round(element["avg-score"].mean(), 2))
    annots.append(np.round(element["annotators"].mean(),2))
    i += 1                   
    
exploration_df = pd.DataFrame([rows, zscores, avgscores, annots]).T.rename(columns={0:"rows", 1:"avg z-score", 2:"avg avg-score", 3:"avg annotators"})
exploration_df["description"] = descriptions
exploration_df = exploration_df.set_index("description")
exploration_df

,rows,avg z-score,avg avg-score,avg annotators
description,,,,
Russian into English,17980.0,0.01,74.50,1.30
German into English,21704.0,0.00,71.85,1.50
Czech into English,11585.0,-0.03,69.24,1.89
Chinese into English,26419.0,-0.05,66.06,1.42
English into Chinese,10221.0,-0.06,65.98,1.58
English into Finish,6748.0,-0.14,45.12,1.23


In [6]:
exploration_df.corr()

,rows,avg z-score,avg avg-score,avg annotators
rows,1.000000,0.597505,0.579839,-0.105454
avg z-score,0.597505,1.000000,0.975645,0.310459
avg avg-score,0.579839,0.975645,1.000000,0.417110
avg annotators,-0.105454,0.310459,0.417110,1.000000


As there are only 6 different types of translations, these correlations might be not very meaningful!

# Lexical metrics

## BLEU Score - Part 1

##### Attention

The BLEU Score might require multiple reference sentences, hence it might not be suitable for our problem!

Inspiration taken from 
* https://towardsdatascience.com/nlp-metrics-made-simple-the-bleu-score-b06b14fbdbc1

In [7]:
from collections import Counter

# a more "pythonic" way to compute BLUE_star 

def BLEU_star_compact(refs, candidate):
    refs = [refs.split()]
    candidate = candidate.split()

    return sum([min(count, max([ref[word] for ref in [Counter(ref) for ref in refs]])) for word, count in Counter(candidate).items()])/len(candidate)

In [8]:
overall_results = {}

correlations_p = []
correlations_k = []

for element in [ru_en, de_en, cs_en, zh_en, en_zh, en_fi]:
    
    bleu_scores = []

    #calculating the bleu scores for the translations in comparison to their respective reference
    for i in range(6748): #element.shape[0]
        reference = element.loc[i,"reference"]
        translation = element.loc[i,"translation"]
        bleu_scores.append(BLEU_star_compact(reference, translation))

    #add the bleu scores to the dataframe
    development_df = element.iloc[:6748,:].copy() #element.shape[0]
    development_df["BLEU"] = bleu_scores
    correlations_p.append(development_df.corr(method="pearson").iloc[-1:,0].values[0])
    correlations_k.append(development_df.corr(method="kendall").iloc[-1:,0].values[0])


print("\033[1mCorrelation between z-score and BLEU score\n")
i = 0
for element in correlations_p:
    print("\033[1m", descriptions[i] + ":",  "\033[0mPearson:", np.round(element,4), "| Kendall:", np.round(correlations_k[i],4))
    i += 1

print("\n\033[1mOverall:\033[0m Average Pearson:", np.round(sum(correlations_p)/len(correlations_p),4),
         "| Average Kendall:", np.round(sum(correlations_k)/len(correlations_k),4))

overall_results["BLEU Star Pearson"] = correlations_p
overall_results["BLEU Star Kendall"] = correlations_k

Correlation between z-score and BLEU score

 Russian into English: Pearson: 0.3198 | Kendall: 0.2168
 German into English: Pearson: 0.2907 | Kendall: 0.2039
 Czech into English: Pearson: 0.4245 | Kendall: 0.2871
 Chinese into English: Pearson: 0.3385 | Kendall: 0.2264
 English into Chinese: Pearson: 0.0308 | Kendall: 0.0116
 English into Finish: Pearson: 0.5084 | Kendall: 0.3383

Overall: Average Pearson: 0.3188 | Average Kendall: 0.214


## BLEU Score - Part 2

### 1st Try (sentence_bleu)

Inspiration taken from 
* https://www.journaldev.com/46659/bleu-score-in-python

In [9]:
from nltk.translate.bleu_score import sentence_bleu

correlations_p = []
correlations_k = []

for element in [ru_en, de_en, cs_en, zh_en, en_zh, en_fi]:
    
    bleu_scores = []

    #calculating the bleu scores for the translations in comparison to their respective reference
    for i in range(6748): #element.shape[0]
        reference = [element.loc[i,"reference"].split()]
        translation = element.loc[i,"translation"].split()
        bleu_scores.append(sentence_bleu(reference, translation,weights=(0.25, 0.25, 0.25, 0.25)))

    #add the bleu scores to the dataframe
    development_df = element.iloc[:6748,:].copy() #element.shape[0]
    development_df["BLEU"] = bleu_scores
    correlations_p.append(development_df.corr(method="pearson").iloc[-1:,0].values[0])
    correlations_k.append(development_df.corr(method="kendall").iloc[-1:,0].values[0])


print("\033[1mCorrelation between z-score and BLEU score\n")
i = 0
for element in correlations_p:
    print("\033[1m", descriptions[i] + ":",  "\033[0mPearson:", np.round(element,4), "| Kendall:", np.round(correlations_k[i],4))
    i += 1

print("\n\033[1mOverall:\033[0m Average Pearson:", np.round(sum(correlations_p)/len(correlations_p),4),
         "| Average Kendall:", np.round(sum(correlations_k)/len(correlations_k),4))

overall_results["BLEU Sentence Pearson"] = correlations_p
overall_results["BLEU Sentence Kendall"] = correlations_k

/Users/franz/.local/lib/python3.8/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/franz/.local/lib/python3.8/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/franz/.local/lib/python3.8/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunc

Correlation between z-score and BLEU score

 Russian into English: Pearson: 0.248 | Kendall: 0.1799
 German into English: Pearson: 0.2415 | Kendall: 0.1698
 Czech into English: Pearson: 0.2831 | Kendall: 0.2227
 Chinese into English: Pearson: 0.2482 | Kendall: 0.1783
 English into Chinese: Pearson: 0.0183 | Kendall: 0.0116
 English into Finish: Pearson: 0.2623 | Kendall: 0.2896

Overall: Average Pearson: 0.2169 | Average Kendall: 0.1753


### 2nd Try (corpus_bleu)

Inspiration taken from:
* https://stackoverflow.com/questions/62337356/bleu-error-n-gram-overlaps-of-lower-order

In [10]:
from nltk.translate.bleu_score import corpus_bleu

correlations_p = []
correlations_k = []

for element in [ru_en, de_en, cs_en, zh_en, en_zh, en_fi]:
    
    bleu_scores = []

    #calculating the bleu scores for the translations in comparison to their respective reference
    for i in range(6748): #element.shape[0]
        reference = [element.loc[i,"reference"].split()]
        translation = element.loc[i,"translation"].split()
        while len(reference) < len(translation):
            reference.append(" ")
        while len(reference) > len(translation):
            translation.append(" ")
        bleu_scores.append(corpus_bleu(reference, translation))

    #add the bleu scores to the dataframe
    development_df = element.iloc[:6748,:].copy() #element.shape[0]
    development_df["BLEU"] = bleu_scores
    correlations_p.append(development_df.corr(method="pearson").iloc[-1:,0].values[0])
    correlations_k.append(development_df.corr(method="kendall").iloc[-1:,0].values[0])


print("\033[1mCorrelation between z-score and BLEU score\n")
i = 0
for element in correlations_p:
    print("\033[1m", descriptions[i] + ":",  "\033[0mPearson", np.round(element,4), "| Kendall:", np.round(correlations_k[i],4))
    i += 1

print("\n\033[1mOverall:\033[0m Average Pearson:", np.round(sum(correlations_p)/len(correlations_p),4),
         "| Average Kendall:", np.round(sum(correlations_k)/len(correlations_k),4))

overall_results["BLEU Corpus Pearson"] = correlations_p
overall_results["BLEU Corpus Kendall"] = correlations_k

Correlation between z-score and BLEU score

 Russian into English: Pearson 0.0346 | Kendall: 0.0563
 German into English: Pearson 0.0355 | Kendall: 0.0536
 Czech into English: Pearson 0.1055 | Kendall: 0.1019
 Chinese into English: Pearson 0.0587 | Kendall: 0.055
 English into Chinese: Pearson 0.4144 | Kendall: 0.2907
 English into Finish: Pearson 0.2355 | Kendall: 0.1723

Overall: Average Pearson: 0.1474 | Average Kendall: 0.1216


## ROUGE Score

Inspiration taken from:
* https://pypi.org/project/rouge-score/
* https://towardsdatascience.com/the-ultimate-performance-metric-in-nlp-111df6c64460

In [11]:
#!pip install --target=/Users/franz/opt/anaconda3/envs/Data_visualization/lib/python3.8/site-packages/ rouge_score

### ROUGE 1

In [12]:
from rouge_score import rouge_scorer

results_p = pd.DataFrame()
results_k = pd.DataFrame()

correlations_p = []
correlations_k = []
j = 0

for element in [ru_en, de_en, cs_en, zh_en, en_zh, en_fi]:
    
    precisions = []
    recalls = []
    fmeasures = []

    #calculating the rouge scores for the translations in comparison to their respective reference
    for i in range(6748): #element.shape[0]
        scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        scores = scorer.score(element.loc[i,"reference"], element.loc[i,"translation"])
        precisions.append(scores["rouge1"].precision)
        recalls.append(scores["rouge1"].recall)
        fmeasures.append(scores["rouge1"].fmeasure)

    #add the rouge scores to the dataframe
    development_df = element.iloc[:6748,:].copy() #element.shape[0]
    development_df["ROUGE precision"] = precisions
    development_df["ROUGE recall"] = recalls
    development_df["ROUGE fmeasure"] = fmeasures
    correlations_p.append(development_df.corr(method="pearson").iloc[-1:,0].values[0])
    correlations_k.append(development_df.corr(method="kendall").iloc[-1:,0].values[0])
    col_name = descriptions[j]
    j += 1
    results_p[col_name] = pd.Series(development_df.corr(method="pearson").iloc[0,3:])
    results_k[col_name] = pd.Series(development_df.corr(method="kendall").iloc[0,3:])


print("\033[1mPearson Correlation between z-score and ROUGE measures (ROUGE 1) \n")


overall_results["ROUGE 1 Precision Pearson"] = results_p.iloc[0,:6].values.tolist()
overall_results["ROUGE 1 Recall Pearson"] = results_p.iloc[1,:6].values.tolist()
overall_results["ROUGE 1 Fmeasure Pearson"] = results_p.iloc[2,:6].values.tolist()

results_p["Average"] = results_p.T.mean()
results_p

Pearson Correlation between z-score and ROUGE measures (ROUGE 1) 



,Russian into English,German into English,Czech into English,Chinese into English,English into Chinese,English into Finish,Average
ROUGE precision,0.333682,0.308229,0.450566,0.360388,0.066999,0.549302,0.344861
ROUGE recall,0.287828,0.299770,0.376656,0.279403,0.065452,0.513220,0.303721
ROUGE fmeasure,0.332984,0.326371,0.440464,0.348995,0.071994,0.544540,0.344225


In [13]:
print("\033[1mKendall Tau Correlation between z-score and ROUGE measures (ROUGE 1) \n")

overall_results["ROUGE 1 Precision Kendall"] = results_k.iloc[0,:6].values.tolist()
overall_results["ROUGE 1 Recall Kendall"] = results_k.iloc[1,:6].values.tolist()
overall_results["ROUGE 1 Fmeasure Kendall"] = results_k.iloc[2,:6].values.tolist()

results_k["Average"] = results_k.T.mean()
results_k

Kendall Tau Correlation between z-score and ROUGE measures (ROUGE 1) 



,Russian into English,German into English,Czech into English,Chinese into English,English into Chinese,English into Finish,Average
ROUGE precision,0.221403,0.218162,0.304000,0.240620,0.044400,0.363198,0.231964
ROUGE recall,0.192943,0.212362,0.249312,0.181416,0.043840,0.332239,0.202019
ROUGE fmeasure,0.223334,0.227899,0.295210,0.229154,0.047778,0.354801,0.229696


### ROUGE 2

In [14]:
from rouge_score import rouge_scorer

results_p = pd.DataFrame()
results_k = pd.DataFrame()

correlations_p = []
correlations_k = []
j = 0

for element in [ru_en, de_en, cs_en, zh_en, en_zh, en_fi]:
    
    precisions = []
    recalls = []
    fmeasures = []

    #calculating the rouge scores for the translations in comparison to their respective reference
    for i in range(6748): #element.shape[0]
        scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        scores = scorer.score(element.loc[i,"reference"], element.loc[i,"translation"])
        precisions.append(scores["rouge2"].precision)
        recalls.append(scores["rouge2"].recall)
        fmeasures.append(scores["rouge2"].fmeasure)

    #add the rouge scores to the dataframe
    development_df = element.iloc[:6748,:].copy() #element.shape[0]
    development_df["ROUGE precision"] = precisions
    development_df["ROUGE recall"] = recalls
    development_df["ROUGE fmeasure"] = fmeasures
    correlations_p.append(development_df.corr(method="pearson").iloc[-1:,0].values[0])
    correlations_k.append(development_df.corr(method="kendall").iloc[-1:,0].values[0])
    col_name = descriptions[j]
    j += 1
    results_p[col_name] = pd.Series(development_df.corr(method="pearson").iloc[0,3:])
    results_k[col_name] = pd.Series(development_df.corr(method="kendall").iloc[0,3:])


print("\033[1mPearson Correlation between z-score and ROUGE measures (ROUGE 2) \n")

overall_results["ROUGE 2 Precision Pearson"] = results_p.iloc[0,:6].values.tolist()
overall_results["ROUGE 2 Recall Pearson"] = results_p.iloc[1,:6].values.tolist()
overall_results["ROUGE 2 Fmeasure Pearson"] = results_p.iloc[2,:6].values.tolist()

results_p["Average"] = results_p.T.mean()
results_p

Pearson Correlation between z-score and ROUGE measures (ROUGE 2) 



,Russian into English,German into English,Czech into English,Chinese into English,English into Chinese,English into Finish,Average
ROUGE precision,0.317852,0.302509,0.403146,0.320802,0.080609,0.468350,0.315545
ROUGE recall,0.298319,0.289925,0.369502,0.290357,0.083000,0.444354,0.295909
ROUGE fmeasure,0.314050,0.302747,0.395101,0.313292,0.086130,0.461915,0.312206


In [15]:
print("\033[1mKendall Tau Correlation between z-score and ROUGE measures (ROUGE 2) \n")

overall_results["ROUGE 2 Precision Kendall"] = results_k.iloc[0,:6].values.tolist()
overall_results["ROUGE 2 Recall Kendall"] = results_k.iloc[1,:6].values.tolist()
overall_results["ROUGE 2 Fmeasure Kendall"] = results_k.iloc[2,:6].values.tolist()

results_k["Average"] = results_k.T.mean()
results_k

Kendall Tau Correlation between z-score and ROUGE measures (ROUGE 2) 



,Russian into English,German into English,Czech into English,Chinese into English,English into Chinese,English into Finish,Average
ROUGE precision,0.219654,0.211056,0.280131,0.214792,0.058583,0.320441,0.217443
ROUGE recall,0.205574,0.203474,0.253486,0.191793,0.059493,0.304527,0.203058
ROUGE fmeasure,0.215878,0.210021,0.270441,0.206967,0.060129,0.314192,0.212938


### ROUGE L

In [16]:
from rouge_score import rouge_scorer

results_p = pd.DataFrame()
results_k = pd.DataFrame()

correlations_p = []
correlations_k = []
j = 0

for element in [ru_en, de_en, cs_en, zh_en, en_zh, en_fi]:
    
    precisions = []
    recalls = []
    fmeasures = []

    #calculating the rouge scores for the translations in comparison to their respective reference
    for i in range(6748): #element.shape[0]
        scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        scores = scorer.score(element.loc[i,"reference"], element.loc[i,"translation"])
        precisions.append(scores["rougeL"].precision)
        recalls.append(scores["rougeL"].recall)
        fmeasures.append(scores["rougeL"].fmeasure)

    #add the rouge scores to the dataframe
    development_df = element.iloc[:6748,:].copy() #element.shape[0]
    development_df["ROUGE precision"] = precisions
    development_df["ROUGE recall"] = recalls
    development_df["ROUGE fmeasure"] = fmeasures
    correlations_p.append(development_df.corr(method="pearson").iloc[-1:,0].values[0])
    correlations_k.append(development_df.corr(method="kendall").iloc[-1:,0].values[0])
    col_name = descriptions[j]
    j += 1
    results_p[col_name] = pd.Series(development_df.corr(method="pearson").iloc[0,3:])
    results_k[col_name] = pd.Series(development_df.corr(method="kendall").iloc[0,3:])


print("\033[1mPearson Correlation between z-score and ROUGE measures (ROUGE L) \n")

overall_results["ROUGE L Precision Pearson"] = results_p.iloc[0,:6].values.tolist()
overall_results["ROUGE L Recall Pearson"] = results_p.iloc[1,:6].values.tolist()
overall_results["ROUGE L Fmeasure Pearson"] = results_p.iloc[2,:6].values.tolist()

results_p["Average"] = results_p.T.mean()
results_p

Pearson Correlation between z-score and ROUGE measures (ROUGE L) 



,Russian into English,German into English,Czech into English,Chinese into English,English into Chinese,English into Finish,Average
ROUGE precision,0.341043,0.317401,0.467481,0.354257,0.068515,0.540008,0.348118
ROUGE recall,0.304441,0.309437,0.402230,0.291957,0.068080,0.505371,0.313586
ROUGE fmeasure,0.340184,0.331321,0.455608,0.346621,0.074154,0.535137,0.347171


In [17]:
print("\033[1mKendall Tau Correlation between z-score and ROUGE measures (ROUGE L) \n")

overall_results["ROUGE L Precision Kendall"] = results_k.iloc[0,:6].values.tolist()
overall_results["ROUGE L Recall Kendall"] = results_k.iloc[1,:6].values.tolist()
overall_results["ROUGE L Fmeasure Kendall"] = results_k.iloc[2,:6].values.tolist()

results_k["Average"] = results_k.T.mean()
results_k

Kendall Tau Correlation between z-score and ROUGE measures (ROUGE L) 



,Russian into English,German into English,Czech into English,Chinese into English,English into Chinese,English into Finish,Average
ROUGE precision,0.231716,0.226267,0.316472,0.239964,0.044429,0.355700,0.235758
ROUGE recall,0.209684,0.220059,0.269153,0.196207,0.044658,0.326671,0.211072
ROUGE fmeasure,0.232707,0.232192,0.305452,0.230285,0.048037,0.347729,0.232734


## RESULTS TABLE

In [18]:
pearsons = []
kendalls = []

for element in list(overall_results.keys()):
    if element.endswith("Pearson"):
        pearsons.append(element)
    elif element.endswith("Kendall"):
        kendalls.append(element)
        
dict_pearson = { your_key: overall_results[your_key] for your_key in pearsons }
dict_kendall = { your_key: overall_results[your_key] for your_key in kendalls }

pearson_df = pd.DataFrame(list(dict_pearson.values()), index=list(dict_pearson.keys()), columns=descriptions)
pearson_df

,Russian into English,German into English,Czech into English,Chinese into English,English into Chinese,English into Finish
BLEU Star Pearson,0.319829,0.290713,0.424481,0.338450,0.030846,0.508383
BLEU Sentence Pearson,0.247967,0.241474,0.283068,0.248230,0.018344,0.262296
BLEU Corpus Pearson,0.034586,0.035531,0.105539,0.058663,0.414448,0.235517
ROUGE 1 Precision Pearson,0.333682,0.308229,0.450566,0.360388,0.066999,0.549302
ROUGE 1 Recall Pearson,0.287828,0.299770,0.376656,0.279403,0.065452,0.513220
ROUGE 1 Fmeasure Pearson,0.332984,0.326371,0.440464,0.348995,0.071994,0.544540
ROUGE 2 Precision Pearson,0.317852,0.302509,0.403146,0.320802,0.080609,0.468350
ROUGE 2 Recall Pearson,0.298319,0.289925,0.369502,0.290357,0.083000,0.444354
ROUGE 2 Fmeasure Pearson,0.314050,0.302747,0.395101,0.313292,0.086130,0.461915
ROUGE L Precision Pearson,0.341043,0.317401,0.467481,0.354257,0.068515,0.540008


In [19]:
pearson_evaluation = pd.DataFrame(pearson_df.idxmax(), columns=["Metric with highest correlation"])
pearson_evaluation["Value"] = pearson_df.max()
pearson_evaluation

,Metric with highest correlation,Value
Russian into English,ROUGE L Precision Pearson,0.341043
German into English,ROUGE L Fmeasure Pearson,0.331321
Czech into English,ROUGE L Precision Pearson,0.467481
Chinese into English,ROUGE 1 Precision Pearson,0.360388
English into Chinese,BLEU Corpus Pearson,0.414448
English into Finish,ROUGE 1 Precision Pearson,0.549302


In [20]:
kendall_df = pd.DataFrame(list(dict_kendall.values()), index=list(dict_kendall.keys()), columns=descriptions)
kendall_df

,Russian into English,German into English,Czech into English,Chinese into English,English into Chinese,English into Finish
BLEU Star Kendall,0.216759,0.203917,0.287051,0.226367,0.011641,0.338281
BLEU Sentence Kendall,0.179918,0.169787,0.222721,0.178306,0.011583,0.289645
BLEU Corpus Kendall,0.056322,0.053605,0.101863,0.054996,0.290729,0.172336
ROUGE 1 Precision Kendall,0.221403,0.218162,0.304000,0.240620,0.044400,0.363198
ROUGE 1 Recall Kendall,0.192943,0.212362,0.249312,0.181416,0.043840,0.332239
ROUGE 1 Fmeasure Kendall,0.223334,0.227899,0.295210,0.229154,0.047778,0.354801
ROUGE 2 Precision Kendall,0.219654,0.211056,0.280131,0.214792,0.058583,0.320441
ROUGE 2 Recall Kendall,0.205574,0.203474,0.253486,0.191793,0.059493,0.304527
ROUGE 2 Fmeasure Kendall,0.215878,0.210021,0.270441,0.206967,0.060129,0.314192
ROUGE L Precision Kendall,0.231716,0.226267,0.316472,0.239964,0.044429,0.355700


In [22]:
kendall_evaluation = pd.DataFrame(kendall_df.idxmax(), columns=["Metric with highest correlation"])
kendall_evaluation["Value"] = kendall_df.max()
kendall_evaluation

,Metric with highest correlation,Value
Russian into English,ROUGE L Fmeasure Kendall,0.232707
German into English,ROUGE L Fmeasure Kendall,0.232192
Czech into English,ROUGE L Precision Kendall,0.316472
Chinese into English,ROUGE 1 Precision Kendall,0.240620
English into Chinese,BLEU Corpus Kendall,0.290729
English into Finish,ROUGE 1 Precision Kendall,0.363198


## COMBINATION